<a href="https://colab.research.google.com/github/sunnyna/TIL/blob/master/08_Prac_2_Keyword_Extraction%EC%9D%98_%EC%A7%81%EC%A0%91_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8장. 핵심 키워드 추출 (Keyword Extraction)

# 8-0 데이터 준비

## Mecab 설치 (필요시)

In [ ]:
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 66 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded,

# 8-1 TF-IDF 활용 핵심키워드 추출

## 실습 1. sklearn 활용


In [ ]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
  res = requests.get(url)
  bs = BeautifulSoup(res.content, 'html.parser')

  title = bs.select('h3#articleTitle')[0].text #제목
  content = bs.select('#articleBodyContents')[0].get_text().replace('\n', " ") #본문
  content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
  return  content.strip()

docs = []
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728') )
len(docs)

### 1) 전처리

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

### 2) TF-IDF 계산

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_df=0.85, max_features=10000)1
word_count_vector = count_vectorizer.fit_transform(preprocessed_docs)
list(count_vectorizer.vocabulary_.keys())[:10]

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

### 3) 핵심키워드 추출

In [ ]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [ ]:
doc = preprocessed_docs[0] # 핵심키워드 추출할 문서 조회

feature_names = count_vectorizer.get_feature_names() # TF-IDF 단어 목록
tf_idf_vector = tfidf_transformer.transform(count_vectorizer.transform([doc])) # 문서의 tf-idf 추출
sorted_keywords = sort_keywords(tf_idf_vector.tocoo()) # TF-IDF를 기준으로 역순 정렬
 
# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)
 
print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)

In [ ]:
tf_idf_vector.tocoo().data


---


## 실습 2. gensim 활용


### 1) 전처리

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

### 2) TF-IDF 계산

In [ ]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

document_ls = [doc.split() for doc in preprocessed_docs]
dct = Dictionary(document_ls) # 인덱스(key) - 단어(valuue) 인 딕셔너리 생성
corpus = [dct.doc2bow(doc) for doc in document_ls] # 각 문서에 포함된 단어를 인덱스로 변환하여 corpus 생성
tfidf = TfidfModel(corpus) # TF-IDF 산출

### 3) 핵심키워드 추출

In [ ]:
def sort_keywords(tfidf):
    return sorted(tfidf, key=lambda x: (x[1], x[0]), reverse=True)

def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [ ]:
doc = corpus[0]

sorted_keywords = sort_keywords(tfidf[doc]) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(dct, sorted_keywords, 5)

print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)

In [ ]:
tfidf[doc]



---



# 8-2 Textrank
https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf

<img src="https://3.bp.blogspot.com/-yp0Lr3ec5EY/XIs6znCcO_I/AAAAAAAAAPY/xtZxe_OYtH0xeuWsp4Qd4DQrunGMpVQmQCLcBGAs/s640/keyword-extraction-textrank.png" />

## 실습 1. 행렬 활용 


### 1) 토큰화 (Tokenization)

분석 텍스트 정제

### 2) Unique한 토큰 목록 생성

그래프 생성을 위해서 Unique한 토큰 목록 생성

In [ ]:
token = ['딸기', '바나나', '사과', '딸기', '파인애플']
nodes = ['바나나', '사과', '파인애플', '딸기']
vocab = nodes

vocab2idx = {vocab[i] : i for i in range(len(vocab))} #vocab을 인덱스로 변환
idx2vocab = {i : vocab[i] for i in range(len(vocab))} #인덱스를 vocab으로 변환
vocab2idx

In [ ]:
len(token)

### 3) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 
*   weighted_edge 행렬은 노드간 가중치 정보를 담고 있음
*   weighted_edge[i][j] 는 i번째 단어와 j번째 단어의 가중치를 의미
*   weighted_edge[i][j] 가 0인 경우는 노드간 연결이 없음을 의미
*   모든 노드는 1로 초기화

In [ ]:
import numpy as np
import math
vocab_len = len(vocab)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len, vocab_len))

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((vocab_len))



# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 2
covered_coocurrences = []

# weighted_edge 구현
for window_start in range(len(token) - window_size + 1):
  window = token[window_start : window_start + window_size]
  for i in range(window_size):
    for j in range(i + 1, window_size ):
      if window[i] in vocab2idx.keys() and window[j] in vocab2idx.keys():
        index_i = window_start + i
        index_j = window_start + j

        if [index_i, index_j] not in covered_coocurrences:
          weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1
          weighted_edge[vocab2idx[window[j]]][vocab2idx[window[i]]] = 1
          covered_coocurrences.append([index_i, index_j])


for i in range(len(vocab)):
  sumi = weighted_edge[i].sum()
  weighted_edge[i] = weighted_edge[i]/sumi if sumi > 0 else 0
  



weighted_edge

### 4) 각 노드의 score계산
각 노드와 연결된 weighted edge의 값을 합산

In [ ]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold
score = np.ones((vocab_len))


for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(len(vocab)):
      summation = 0
      for j in range(len(vocab)):
        if weighted_edge[j][i] != 0:
          summation += weighted_edge[j][i] * score[j]

        score[i] = (1 -d) + d * (summation)

      if np.sum(np.fabs(prev_score - score)) <= threshold:
        break
score

In [ ]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold
score = np.ones((vocab_len))


for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(len(vocab)):
      summation = 0
      for j in range(len(vocab)):
        if weighted_edge[j][i] != 0:
          summation += weighted_edge[j][i] * score[j]

        score[i] = (1 -d) + d * (summation)

      if np.sum(np.fabs(prev_score - score)) <= threshold:
        break
score


### 5) 핵심 단어 추출

In [ ]:
sorted_index = None

sorted_index = (np.argsort(score)[::-1])
n = 4
print("\n=== 핵심키워드 ===")
for i in range(n):
    print(str(idx2vocab[sorted_index[i]])+" : " + str(score[sorted_index[i]]))

In [ ]:
sorted_index[0]

---

In [ ]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
  res = requests.get(url)
  bs = BeautifulSoup(res.content, 'html.parser')

  title = bs.select('h3#articleTitle')[0].text #제목
  content = bs.select('#articleBodyContents')[0].get_text().replace('\n', " ") #본문
  content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
  return  content.strip()

doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc[:50]

'과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 510'

  ### 1) 토큰화 (Tokenization)

  분석 텍스트 정제

In [ ]:
  from konlpy.tag import Mecab
  mecab = Mecab()

  nodes = [token for token in mecab.pos(doc) if token[1] in ['NNG','NNP']] #NNG, NNP를 스코어 계산 대상(노드)로 제한
  print(nodes)
  tokens = [token for token in mecab.pos(doc)] #탐색할 토큰 전체
  print(tokens)

[('과기', 'NNG'), ('정통부', 'NNG'), ('유영민', 'NNP'), ('장관', 'NNG'), ('참석', 'NNG'), ('기념행사', 'NNG'), ('투입', 'NNG'), ('여종', 'NNG'), ('데이터', 'NNG'), ('구축', 'NNG'), ('민간', 'NNG'), ('클라우드', 'NNP'), ('외부', 'NNG'), ('연계', 'NNG'), ('체계', 'NNG'), ('개방', 'NNG'), ('강화', 'NNG'), ('데일리', 'NNP'), ('이재운', 'NNP'), ('기자', 'NNG'), ('국가', 'NNG'), ('차원', 'NNG'), ('빅', 'NNG'), ('데이터', 'NNG'), ('활용', 'NNG'), ('시대', 'NNG'), ('산업', 'NNG'), ('창출', 'NNG'), ('기존', 'NNG'), ('산업', 'NNG'), ('변화', 'NNG'), ('혁신', 'NNG'), ('장', 'NNG'), ('센터', 'NNG'), ('문', 'NNG'), ('분야', 'NNG'), ('데이터', 'NNG'), ('경제', 'NNG'), ('발전', 'NNG'), ('정부', 'NNG'), ('청사진', 'NNG'), ('현실', 'NNG'), ('구현', 'NNG'), ('계획', 'NNG'), ('과학', 'NNG'), ('기술', 'NNG'), ('정보', 'NNG'), ('통신부', 'NNG'), ('서울', 'NNP'), ('중구', 'NNP'), ('대한', 'NNP'), ('상공', 'NNG'), ('회의소', 'NNG'), ('데이터', 'NNG'), ('생태', 'NNG'), ('조성', 'NNG'), ('혁신', 'NNG'), ('성장', 'NNG'), ('기반', 'NNG'), ('마련', 'NNG'), ('빅', 'NNG'), ('데이터', 'NNG'), ('플랫', 'NNG'), ('폼', 'NNG'), ('센터', 'NNG'), ('출범식', 'NNG'

### 2) 그래프 생성 (weighted edge 계산)

In [ ]:
import networkx as nx

def connect(nodes, tokens):
  window_size = 3

  edges = []
  for window_start in range(len(tokens) - window_size + 1):
    window = tokens[window_start : window_start + window_size]
    for i in range(window_size):
      for j in range(i + 1, window_size):
        if (window[i] in nodes) & (window[j] in nodes ):
          edges.append([window[i], window[j]])

  return edges

graph = nx.diamond_graph()
graph.clear()
graph.add_nodes_from(list(set(nodes)))
graph.add_edges_from(connect(nodes, tokens))
graph

### 3) 각 노드의 score계산

In [ ]:
scores = nx.pagerank(graph)

rank = sorted(scores.items(), key=lambda x: x[1], reverse=True)
rank[:5]

[(('데이터', 'NNG'), 0.04440161015091129),
 (('센터', 'NNG'), 0.01637352097715778),
 (('한국', 'NNP'), 0.012528472823809035),
 (('경제', 'NNG'), 0.011955384156603086),
 (('정보', 'NNG'), 0.01095187571871316)]

### 4)핵심 단어 추출

In [ ]:
rank = rank[:100]

## 실습 3. TextRank 핵심 구 추출

### 1) 명사를 기준으로 구 추출


In [ ]:
phrases = []
phrase  = ' '
for word in tokens:
  if word[1][0] == 'N':
    phrase += str(word[0])
    phrase += " "
  else:
    if phrase != ' ':
      phrases.append(phrase.strip())
    phrase = ' '

unique_phrases = list(set(phrases))
print(unique_phrases[:10])

['기관', '이용자', '의료비 절감', '민간', '문화 플랫 폼', '영상', '개', '초석', '년 간', '예술 관람']


### 2) 각 구의 Score 계산

In [ ]:
vocabs = dict((r[0][0], r[1]) for r in rank)

phrase_scores = []
keywords = []
for phrase in unique_phrases:
  phrase_score = 0
  for word in phrase.split():
    if word in vocabs.keys():
        phrase_score += vocabs[word.strip()]
        
  phrase_scores.append(phrase_score)
  keywords.append(phrase) 

for i, keyword in enumerate(keywords[:10]):
  print("Keyword : {} , Score : {}".format(keyword, phrase_scores[i]))

Keyword : 기관 , Score : 0.0029394578260479864
Keyword : 이용자 , Score : 0.0017882366614242664
Keyword : 의료비 절감 , Score : 0.010465871834157876
Keyword : 민간 , Score : 0.007075709402775881
Keyword : 문화 플랫 폼 , Score : 0.023732687445707147
Keyword : 영상 , Score : 0.0006425358749196831
Keyword : 개 , Score : 0
Keyword : 초석 , Score : 0.0006425358749196831
Keyword : 년 간 , Score : 0.0006425358749196831
Keyword : 예술 관람 , Score : 0.005779708983172633


### 3) 각 구를 Score로 정렬하여 핵심 구 추출

In [ ]:
import numpy as np
sorted_index = np.argsort(phrase_scores)[::-1]
for i in range(10):
  print(keywords[sorted_index[i]])

여종 데이터 구축 민간 클라우드
빅 데이터 플랫 폼
빅 데이터 센터
개 빅 데이터 센터
빅 데이터 활용 시대
민간 사이 데이터 파일 형식 등
빅 데이터 활용
픽사 베이 빅 데이터
여종 데이터 구축
데이터 경제


## 실습 4. gensim 활용

In [ ]:
from gensim.summarization import keywords
keywords(doc, words=5).split('\n')

['데이터', '데이터를', '계획이다', '빅데이터', '문재인']

In [ ]:
keywords?